# Trusted Zone — Documents Processing

This notebook handles the **document processing** step for the Trusted Zone of our data pipeline.  
Its primary goal is to:

1. **Load recipe IDs** that have images (from the images processing step)
2. **Stream-filter the recipe documents** to keep only those with images
3. **Use multipart uploads** to handle large JSONL files efficiently
4. **Generate a processing report** for audit and monitoring

This notebook works in conjunction with `images.ipynb` to ensure data integrity in the Trusted Zone.


## 1. Setup and Configuration


In [ ]:
import os, io, json, re
from pathlib import PurePosixPath
from datetime import datetime, timezone
from typing import Dict, List, Set, Iterable

import boto3
from botocore.config import Config
from botocore.exceptions import ClientError
from dotenv import load_dotenv

load_dotenv()

# S3 / MinIO Configuration
MINIO_USER     = os.getenv("MINIO_USER")
MINIO_PASSWORD = os.getenv("MINIO_PASSWORD")
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT")

session = boto3.session.Session(
    aws_access_key_id=MINIO_USER,
    aws_secret_access_key=MINIO_PASSWORD,
    region_name="us-east-1"
)
s3 = session.client(
    "s3",
    endpoint_url=MINIO_ENDPOINT,
    config=Config(signature_version="s3v4", s3={"addressing_style": "path"})
)

# Paths and Buckets
FORM_BUCKET         = "formatted-zone"
FORM_DOCS_KEY       = "documents/recipes.jsonl"

TRUST_BUCKET        = "trusted-zone"
TRUST_DOCS_KEY      = "documents/recipes.jsonl"
TRUST_REPORT_PREFIX = "reports"

# Input file from images processing
RECIPE_IDS_FILE = "recipe_ids_with_images.json"

# Behavior flags
DRY_RUN   = False
OVERWRITE = True

def utc_ts():
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H-%M-%SZ")


In [ ]:
# --- Documents config & helpers ---

import math
import pandas as pd
import re
from unidecode import unidecode

# Text cleaning config
import nltk
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
LANG = "english"
STOPWORDS = set(stopwords.words(LANG))

def clean_text(s: str) -> str:
    if not s:
        return ""
    s = s.lower()
    s = unidecode(s)
    s = re.sub(r"[^\w\s]", " ", s)         # remove punctuation
    s = re.sub(r"\s+", " ", s).strip()
    toks = [t for t in s.split() if t not in STOPWORDS]
    return " ".join(toks)

def join_text_list(objs, key="text"):
    if not isinstance(objs, list):
        return ""
    return " ".join([str(o.get(key, "")) for o in objs if isinstance(o, dict)])

# Nutritional fields we will consider (adjust if your schema differs)
# Case A (totals per 100g): dict under key like nutr_values_per100g__from_recipes_with_nutritional_info
NUTR_TOTALS_KEY_CANDIDATES = [
    "nutr_values_per100g__from_recipes_with_nutritional_info",
    "nutr_values_per100g",
    "nutr_values",
]
NUTR_TOTALS_NUMERIC_FIELDS = ["energy", "fat", "protein", "salt", "saturates", "sugars"]

# Case B (per-ingredient): array of dicts under key like nutr_per_ingredient__from_recipes_with_nutritional_info
NUTR_PER_INGR_KEY_CANDIDATES = [
    "nutr_per_ingredient__from_recipes_with_nutritional_info",
    "nutr_per_ingredient",
]

# Policy: if per-ingredient exists, we will drop totals to avoid duplication.
DROP_TOTALS_IF_PER_INGR_PRESENT = True

# Helper: pick first existing key
def pick_first_present(rec: dict, candidates: list[str]) -> str | None:
    for k in candidates:
        if k in rec and rec[k] is not None:
            return k
    return None

# IQR helper
def iqr_bounds(vals: list[float], k: float = 1.5):
    if not vals:
        return None
    s = pd.Series(vals)
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    lo, hi = float(q1 - k * iqr), float(q3 + k * iqr)
    return {"q1": float(q1), "q3": float(q3), "lo": lo, "hi": hi}



# Stats accumulator for the report
doc_stats = {
    "total_read": 0,
    "kept_after_id": 0,
    "dropped_missing_nutrition": 0,
    "dropped_outliers": 0,
    "written": 0,
    "nutrition_totals_dropped_due_to_duplication": 0,
    "text_tokens_avg": {
        "ingredients_raw": 0.0, "ingredients_clean": 0.0,
        "instructions_raw": 0.0, "instructions_clean": 0.0,
        "title_raw": 0.0, "title_clean": 0.0,
    }
}

def token_len(s: str) -> int:
    return len((s or "").split())


## 2. Load Recipe IDs from Images Processing

We load the recipe IDs that have images, which were generated by the `images.ipynb` notebook. This creates the coupling between the two processing steps while maintaining clean separation.


In [ ]:
def load_recipe_ids_with_images() -> Set[str]:
    """Load recipe IDs that have images from the images processing step."""
    try:
        with open(RECIPE_IDS_FILE, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        recipe_ids = set(data.get('recipe_ids_with_images', []))
        print(f"[INFO] Loaded {len(recipe_ids)} recipe IDs with images")
        print(f"[INFO] Source: {data.get('source', 'unknown')}")
        print(f"[INFO] Generated at: {data.get('timestamp', 'unknown')}")
        
        return recipe_ids
        
    except FileNotFoundError:
        print(f"[ERROR] Recipe IDs file not found: {RECIPE_IDS_FILE}")
        print("[INFO] Make sure to run images.ipynb first to generate the recipe IDs")
        return set()
    except Exception as e:
        print(f"[ERROR] Failed to load recipe IDs: {e}")
        return set()

# Load the recipe IDs that have images
recipe_ids_with_images = load_recipe_ids_with_images()

if not recipe_ids_with_images:
    print("[ERROR] No recipe IDs with images found. Cannot proceed with document filtering.")
    raise SystemExit("No recipe IDs with images found")


In [ ]:
print("Pre-scanning documents to compute IQR thresholds for nutritional fields...")

def read_jsonl_lines(bucket: str, key: str):
    """Stream JSONL lines from S3."""
    obj = s3.get_object(Bucket=bucket, Key=key)
    for raw in obj["Body"].iter_lines():
        if raw:  # skip empty
            yield raw

nutr_values_collections = {f: [] for f in NUTR_TOTALS_NUMERIC_FIELDS}
total_prescan = 0

for raw in read_jsonl_lines(FORM_BUCKET, FORM_DOCS_KEY):
    try:
        rec = json.loads(raw)
    except Exception:
        continue
    total_prescan += 1

    # Only consider records we *might* keep later: ids_with_images
    rid = rec.get("id")
    if not rid or rid not in recipe_ids_with_images:
        continue

    totals_key = pick_first_present(rec, NUTR_TOTALS_KEY_CANDIDATES)
    if not totals_key:
        continue
    totals = rec.get(totals_key) or {}
    # Collect numeric fields if present and finite
    for f in NUTR_TOTALS_NUMERIC_FIELDS:
        v = totals.get(f)
        if isinstance(v, (int, float)) and math.isfinite(v):
            nutr_values_collections[f].append(float(v))

# Compute bounds
iqr_thresholds = {}
for f, vals in nutr_values_collections.items():
    iqr_thresholds[f] = iqr_bounds(vals) if vals else None

print("IQR thresholds:", iqr_thresholds)

## 3. Multipart JSONL Writer for Large Files

The recipes dataset (`recipes.jsonl`) can be extremely large. Instead of loading everything into memory, we use a streaming approach with multipart uploads to handle the filtering efficiently.


In [ ]:
MIN_PART_SIZE = 8 * 1024 * 1024  # 8 MB

class MultipartJSONLWriter:
    """Streaming JSONL writer that uses S3 multipart uploads for large files."""
    
    def __init__(self, bucket: str, key: str, content_type="application/x-ndjson", metadata=None):
        self.bucket = bucket
        self.key = key
        self.buf = io.BytesIO()
        self.parts = []
        self.part_num = 1
        self.open = True
        extra = {
            "Bucket": bucket,
            "Key": key,
            "ContentType": content_type,
            "Metadata": metadata or {},
        }
        resp = s3.create_multipart_upload(**extra)
        self.upload_id = resp["UploadId"]

    def _flush_part(self):
        """Flush the current buffer as a multipart upload part."""
        self.buf.seek(0)
        body = self.buf.read()
        if not body:
            self.buf.seek(0)
            self.buf.truncate(0)
            return
        resp = s3.upload_part(
            Bucket=self.bucket, Key=self.key,
            UploadId=self.upload_id, PartNumber=self.part_num, Body=body
        )
        self.parts.append({"ETag": resp["ETag"], "PartNumber": self.part_num})
        self.part_num += 1
        self.buf.seek(0); self.buf.truncate(0)

    def write_line(self, raw_line_bytes: bytes):
        """Write a line to the buffer, flushing when buffer is full."""
        # raw_line_bytes is already one JSON object line (no trailing \n required)
        self.buf.write(raw_line_bytes)
        self.buf.write(b"\n")
        if self.buf.tell() >= MIN_PART_SIZE:
            self._flush_part()

    def close(self):
        """Close the writer and complete the multipart upload."""
        if not self.open:
            return
        try:
            # If there's leftover data, flush as a last part
            self._flush_part()
            if not self.parts:
                # No data kept: abort multipart, optionally create empty object
                s3.abort_multipart_upload(
                    Bucket=self.bucket, Key=self.key, UploadId=self.upload_id
                )
                # Optional: write a 0-byte file so the path exists
                s3.put_object(
                    Bucket=self.bucket, Key=self.key, Body=b"",
                    ContentType="application/x-ndjson",
                    Metadata={"note": "empty after filtering", "ts": utc_ts()},
                )
            else:
                s3.complete_multipart_upload(
                    Bucket=self.bucket, Key=self.key, UploadId=self.upload_id,
                    MultipartUpload={"Parts": self.parts}
                )
        finally:
            self.open = False




## 4. Stream-Filter Documents by Recipe IDs

Now we filter the recipe documents to keep only those that have corresponding images. This step ensures that every recipe in the Trusted Zone can be paired with one or more valid images.


In [ ]:
## 4. Stream-Filter + Transform to Trusted Zone

def filter_docs_to_trusted_by_ids():
    """Keep only recipes with images; drop missing/outlier nutrition; clean text; drop duplicated totals."""
    total = kept_after_id = written = 0

    if DRY_RUN:
        print("[DRY_RUN] Counting documents that would be kept after id+nutrition checks...")
        for raw in read_jsonl_lines(FORM_BUCKET, FORM_DOCS_KEY):
            total += 1
            try:
                rec = json.loads(raw)
            except Exception:
                continue

            rid = rec.get("id")
            if not rid or rid not in recipe_ids_with_images:
                continue


            # Missing nutrition check (require at least totals OR per-ingredient)
            totals_key = pick_first_present(rec, NUTR_TOTALS_KEY_CANDIDATES)
            per_ingr_key = pick_first_present(rec, NUTR_PER_INGR_KEY_CANDIDATES)
            if not totals_key and not per_ingr_key:
                continue  # missing nutrition entirely

            # Outlier check on totals if present and thresholds exist
            if totals_key and iqr_thresholds and any(iqr_thresholds.values()):
                totals = rec.get(totals_key) or {}
                outlier = False
                for f in NUTR_TOTALS_NUMERIC_FIELDS:
                    thr = iqr_thresholds.get(f)
                    v = totals.get(f)
                    if thr and isinstance(v, (int, float)) and math.isfinite(v):
                        if not (thr["lo"] <= float(v) <= thr["hi"]):
                            outlier = True
                            break
                if outlier:
                    continue

            kept_after_id += 1

        print(f"[DRY_RUN] total={total} kept_after_id={kept_after_id}")
        return total, kept_after_id, 0

    print("Filtering and transforming documents to Trusted Zone...")

    writer = MultipartJSONLWriter(
        TRUST_BUCKET, TRUST_DOCS_KEY,
        content_type="application/x-ndjson",
        metadata={"note": "ids with images + nutrition checks + text clean + de-dup nutrition", "ts": utc_ts()}
    )

    try:
        for raw in read_jsonl_lines(FORM_BUCKET, FORM_DOCS_KEY):
            doc_stats["total_read"] += 1
            total += 1
            try:
                rec = json.loads(raw)
            except Exception:
                continue

            rid = rec.get("id")
            if not rid or rid not in recipe_ids_with_images:
                continue

            # Require some nutrition info (either totals or per-ingredient)
            # totals_key = pick_first_present(rec, NUTR_TOTALS_KEY_CANDIDATES)
            # per_ingr_key = pick_first_present(rec, NUTR_PER_INGR_KEY_CANDIDATES)
            # if not totals_key and not per_ingr_key:
            #     doc_stats["dropped_missing_nutrition"] += 1
            #     continue

            # Check nutrition info but don't drop recipes without it
            totals_key = pick_first_present(rec, NUTR_TOTALS_KEY_CANDIDATES)
            per_ingr_key = pick_first_present(rec, NUTR_PER_INGR_KEY_CANDIDATES)
            has_nutrition = bool(totals_key or per_ingr_key)
            rec["has_nutrition_data"] = has_nutrition

            # If totals exist, apply outlier filter per field
            outlier = False
            if totals_key and iqr_thresholds and any(iqr_thresholds.values()):
                totals = rec.get(totals_key) or {}
                for f in NUTR_TOTALS_NUMERIC_FIELDS:
                    thr = iqr_thresholds.get(f)
                    v = totals.get(f)
                    if thr and isinstance(v, (int, float)) and math.isfinite(v):
                        val = float(v)
                        if not (thr["lo"] <= val <= thr["hi"]):
                            outlier = True
                            break
            if outlier:
                doc_stats["dropped_outliers"] += 1
                continue

            doc_stats["kept_after_id"] += 1

            # Text cleaning for embeddings — add cleaned siblings without overwriting
            title_raw = rec.get("title__from_layer1") or rec.get("title__from_recipes_with_nutritional_info") or rec.get("title") or ""
            ingr_raw  = join_text_list(rec.get("ingredients__from_layer1")) or join_text_list(rec.get("ingredients__from_recipes_with_nutritional_info")) or join_text_list(rec.get("ingredients")) or ""
            instr_raw = join_text_list(rec.get("instructions__from_layer1")) or join_text_list(rec.get("instructions__from_recipes_with_nutritional_info")) or join_text_list(rec.get("instructions")) or ""

            rec["title_text_raw"] = title_raw
            rec["ingredients_text_raw"] = ingr_raw
            rec["instructions_text_raw"] = instr_raw

            rec["title_text_clean"] = clean_text(title_raw)
            rec["ingredients_text_clean"] = clean_text(ingr_raw)
            rec["instructions_text_clean"] = clean_text(instr_raw)

            # Update running token-length averages (simple incremental mean)
            for k, v in {
                "ingredients_raw": token_len(rec["ingredients_text_raw"]),
                "ingredients_clean": token_len(rec["ingredients_text_clean"]),
                "instructions_raw": token_len(rec["instructions_text_raw"]),
                "instructions_clean": token_len(rec["instructions_text_clean"]),
                "title_raw": token_len(rec["title_text_raw"]),
                "title_clean": token_len(rec["title_text_clean"]),
            }.items():
                # incremental average
                n = max(1, doc_stats["kept_after_id"])
                prev = doc_stats["text_tokens_avg"][k]
                doc_stats["text_tokens_avg"][k] = prev + (v - prev) / n

            # Remove duplicated totals if per-ingredient available (policy)
            if DROP_TOTALS_IF_PER_INGR_PRESENT and per_ingr_key and totals_key:
                if totals_key in rec:
                    rec.pop(totals_key, None)
                    doc_stats["nutrition_totals_dropped_due_to_duplication"] += 1
                rec["nutrition_normalized"] = True
            else:
                rec["nutrition_normalized"] = bool(per_ingr_key and not totals_key)

            # Write transformed record
            writer.write_line(json.dumps(rec, ensure_ascii=False).encode("utf-8"))
            written += 1
            doc_stats["written"] += 1

    finally:
        writer.close()

    print(f"[OK] wrote filtered+transformed docs to s3://{TRUST_BUCKET}/{TRUST_DOCS_KEY}")
    return total, doc_stats["kept_after_id"], written

# Execute the filtering
total_docs, kept_docs, written_docs = filter_docs_to_trusted_by_ids()
print(f"[STATS] total_read={total_docs} kept_after_id={kept_docs} written={written_docs}")
print(f"[STATS] dropped_missing_nutrition={doc_stats['dropped_missing_nutrition']} dropped_outliers={doc_stats['dropped_outliers']}")


## 5. Generate Processing Report

Finally, we generate a comprehensive report of the document processing step and save it to the Trusted Zone for audit and monitoring purposes.


In [ ]:
## 5. Generate Processing Report (expanded)

report = {
    "timestamp": utc_ts(),
    "processing_step": "documents",
    "source_docs": f"s3://{FORM_BUCKET}/{FORM_DOCS_KEY}",
    "destination_docs": f"s3://{TRUST_BUCKET}/{TRUST_DOCS_KEY}",
    "recipe_ids_source": RECIPE_IDS_FILE,

    # Counts
    "total_doc_count": total_docs,
    "kept_after_id": kept_docs,
    "written_doc_count": written_docs,
    "dropped_missing_nutrition": doc_stats["dropped_missing_nutrition"],
    "dropped_outliers": doc_stats["dropped_outliers"],
    "unique_recipe_ids_with_images": len(recipe_ids_with_images),
    "filtering_rate": f"{(written_docs/total_docs*100):.2f}%" if total_docs > 0 else "0%",


    # Nutrition policy & thresholds
    "nutrition": {
        "totals_key_candidates": NUTR_TOTALS_KEY_CANDIDATES,
        "per_ingredient_key_candidates": NUTR_PER_INGR_KEY_CANDIDATES,
        "drop_totals_if_per_ingredient_present": DROP_TOTALS_IF_PER_INGR_PRESENT,
        "iqr_thresholds": iqr_thresholds
    },

    # Text cleaning summary
    "text_cleaning": {
        "lang": LANG,
        "stopwords_count": len(STOPWORDS),
        "token_len_avg": doc_stats["text_tokens_avg"]
    },

    # Effects
    "nutrition_totals_dropped_due_to_duplication": doc_stats["nutrition_totals_dropped_due_to_duplication"],

    # Flags
    "dry_run": DRY_RUN,
    "overwrite": OVERWRITE
}

if not DRY_RUN:
    s3.put_object(
        Bucket=TRUST_BUCKET,
        Key=f"{TRUST_REPORT_PREFIX}/documents_processing_{utc_ts()}.json",
        Body=json.dumps(report, ensure_ascii=False, indent=2).encode("utf-8"),
        ContentType="application/json"
    )
    print(f"[OK] wrote report -> s3://{TRUST_BUCKET}/{TRUST_REPORT_PREFIX}/")
else:
    print("[DRY_RUN] report:", json.dumps(report, indent=2))
